In [1]:
import os
import numpy as np
from osgeo import gdal
import glob
import shelve
import isce
import isceobj
import multiprocessing
import subprocess
import sys
import time
from pathlib import Path
n_processes = 4 #int(multiprocessing.cpu_count()) - 2;


Add required paths

In [38]:
#get the path of the ISCE base dir
env_str = os.popen("conda env list | grep '/isce$' | awk '{print $NF}'")
isce_base_dir = env_str.read().strip();
isce2_share_dir = f"{isce_base_dir}/share/isce2";

# now add stripmapStack contrib directory to PATH 
stripmapStack_dir = f"{isce2_share_dir}/stripmapStack";
# add UAVSAR functionalities to the path
uavsar_utils_dir = os.getenv("HOME")+'/'+'bhuvan_uavsar/uavsar_utils'
os.environ['PATH'] = f"{uavsar_utils_dir}:{stripmapStack_dir}:{os.environ['PATH']}"
os.environ['ATBD'] = '/home/jovyan/atbd-se-development/'
sys.path.append(os.environ['ATBD'])
!export OMP_NUM_THREADS=16

In [3]:
try:
    from uavsar_utils.insar import ImageReader, Interferogram, correlation
except:
    print('Add uavsar_utils directory to pythonpath')


In [4]:
def run_cmd(cmd):
    print("Command: {}".format(cmd))
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print("Command failed with error: {}".format(e))

Put all your inputs here

In [9]:
scratch_dir = '/scratch/bvarugu'; #define your scratch dir
track_name = 'track_14003' #define the UAVSAR directory name
segments = [1,2,3,4]; #define the SLC segments; Used for concatenation
#reference_date = #input a reference date if not the first date will be considered reference date

rlks = 3 #define range looks
alks = 12  #define azimuth looks
numConnections = 2 #define number of interferogram pairs to be made with each SLC similar to ISCE
#bbox = '35 37 -122 -120' #Bounding box for DEM download #selected based on Northern San Andreas

out_dir = os.path.join(scratch_dir,track_name);
os.makedirs(out_dir, exist_ok=True);
download_dir= os.path.join(out_dir,'download');
os.makedirs(download_dir, exist_ok=True);
os.chdir(download_dir);
#os.chdir(out_dir);

In [5]:
#copy the links from UAVSAR webpage into a file in the download dir named download_links.sh
#this cell shall be updated if a better method of downloading UAVSAR data is available
print('Downloading UAVSAR SLCs using wget links')
!sh download_links.sh > download_log
!rm *.ann.*
!rm *.dop.*
os.chdir(out_dir);

--2024-06-06 22:51:22--  https://downloaduav2.jpl.nasa.gov/Release23/SanAnd_14003_02/SanAnd_14003_02_BC.dop
Resolving downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244855 (239K)
Saving to: ‘SanAnd_14003_02_BC.dop’

100%[======================================>] 244,855      234KB/s   in 1.0s   

2024-06-06 22:51:27 (234 KB/s) - ‘SanAnd_14003_02_BC.dop’ saved [244855/244855]

--2024-06-06 22:51:27--  https://downloaduav2.jpl.nasa.gov/Release23/SanAnd_14003_02/SanAnd_14003_02_BC_s1_2x8.llh
Resolving downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 485480580 (463M)
Saving to: ‘SanAnd_14003_02_BC_s1_2x8.llh’

100%[====================

In [ ]:
#Create SLC directory
dop_file = glob.glob(os.path.join(download_dir, '*.dop'))[0];

if len(segments)>1:
    print('Multiple Segments exist. Creating multiple SLC segments directories!')
    for i in range(len(segments)):
        slc_seg_dir = 'SLC_seg{}'.format(segments[i]);
        cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_seg_dir,segments[i]);
        print(cmd);
        os.system(cmd);
        if i< len(segments)-1:
            cmd = 'cp {}/*/*.ann {}'.format(slc_seg_dir,download_dir);
            print(cmd);
            os.system(cmd);
    slc_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0])); 
else:
    print('Procesing only One Segment. No Concatenation')
    slc_dir = os.path.join(out_dir,'SLC');
    cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_dir,segments[0]);
    os.system(cmd);

print('SLC directory:',slc_dir);
dateList = sorted(os.listdir(slc_dir));
print('SLC dates:',dateList);


In [10]:
slc_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0])); 
try:
    reference_date
except:
    reference_date=sorted(os.listdir(slc_dir))[0];
print('Reference date for Meta data:',reference_date);
reference_slc_dir = os.path.join(slc_dir,reference_date);

Reference date for Meta data: 20091118


In [37]:
#read necessary info from the UAVSAR annotation file. Each annotation file contains the info for all the segments.
def read_uavsar_ann_file(ann_file_path):
    '''read UAVSAR ann file as a dict to query for rows, columns info '''
    result_dict = {}
    with open(ann_file_path, 'r') as file:
        for line in file:
            # Strip any leading/trailing whitespace and skip empty lines
            line = line.strip()
            if not line or line.startswith(';'):
                continue
            
            # Split the line into key and value using '=' as the separator
            if '=' in line:
                key, value = line.split('=', 1);
                key = key.split('(')[0].strip()
                value = value.split(';')[0].strip()
                result_dict[key.strip()] = value.strip()
    
    return result_dict
ann_file_path = glob.glob(os.path.join(reference_slc_dir, '*.ann'))[0];
slc_info_dict = read_uavsar_ann_file(ann_file_path);
key = 'slc_{}_1x1 Columns'.format(str(segments[0]))
samples = int(slc_info_dict[key]);
lines = 0;
for seg in segments:
    key = 'slc_{}_1x1 Rows'.format(str(seg))
    lines += int(slc_info_dict[key]);
print('total samples/columns:',samples);
print('total rows:',lines) 
lats= [];lons= [];
try:
    bbox
except:
    for seg in segments:
        for corner in [1,2,3,4]:
            lat, lon = slc_info_dict['Segment {} Data Approximate Corner 1'.format(seg,corner)].split(',')
            lats.append(float(lat));lons.append(float(lon));
    bbox = ' '.join(map(str,[int(np.min(lats)-0.1),int(np.max(lats)+0.1),int(np.min(lons)-0.1),int(np.max(lons)+0.1)]));
print('Bounding box for DEM:',bbox)

total samples/columns: 9709
total rows: 243560
Bounding box for DEM: 35 36 -121 -120


In [39]:
#get DEM for geocoding
#if you have your DEM file deifne its path as dem_file '/path/to/dem_file' if not SRTM DEM will be used
#define bounding box for the DEM
try:
    dem_file
except:
    dem_dir = os.path.join(out_dir,'DEM');
    os.makedirs(dem_dir, exist_ok=True);
    os.chdir(dem_dir);
    source_link ="http://step.esa.int/auxdata/dem/SRTMGL1/"
    try:
        bbox
    except:
        print('Bounding box for DEM download required');
    cmd ='dem.py -a stitch -b {} -u {} -d {} -r -s 1 -c'.format(bbox,source_link,dem_dir);
    print(cmd);
    run_cmd(cmd);
    
dem_file = glob.glob(os.path.join(dem_dir, "*.dem.wgs84"))[0];
cmd = 'fixImageXml.py -f -i {}'.format(dem_file)
print(cmd);
os.system(cmd);
os.chdir(out_dir);

dem.py -a stitch -b 35 36 -121 -120 -u http://step.esa.int/auxdata/dem/SRTMGL1/ -d /scratch/bvarugu/track_14003/DEM -r -s 1 -c
Command: dem.py -a stitch -b 35 36 -121 -120 -u http://step.esa.int/auxdata/dem/SRTMGL1/ -d /scratch/bvarugu/track_14003/DEM -r -s 1 -c


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.3M  100 10.3M    0     0  4463k      0  0:00:02  0:00:02 --:--:-- 4466k


API open (R): /scratch/bvarugu/track_14003/DEM/demLat_N35_N36_Lon_W121_W120.dem
API close:  /scratch/bvarugu/track_14003/DEM/demLat_N35_N36_Lon_W121_W120.dem
GDAL open (R): /scratch/bvarugu/track_14003/DEM/demLat_N35_N36_Lon_W121_W120.dem.vrt
API open (WR): demLat_N35_N36_Lon_W121_W120.dem.wgs84

    << Geoid Correction I2 SRTM>>

Jet Propulsion Laboratory - Radar Science and Engineering


Sampling Geoid at grid points -  Longitude Samples:    13 Latitude Lines:    13
Corner Geoid Heights (m) =  -33.86 -36.78 -32.91 -33.88

Correcting data to geoid height...

At line:      512
At line:     1024
At line:     1536
At line:     2048
At line:     2560
At line:     3072
At line:     3584
GDAL close: /scratch/bvarugu/track_14003/DEM/demLat_N35_N36_Lon_W121_W120.dem.vrt
API close:  demLat_N35_N36_Lon_W121_W120.dem.wgs84
API open (R): demLat_N35_N36_Lon_W121_W120.dem.wgs84
API close:  demLat_N35_N36_Lon_W121_W120.dem.wgs84
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -

In [38]:
#create pairs
looks = [alks, rlks];
ifg_dir =  os.path.join(out_dir,'Igrams');
os.makedirs(ifg_dir, exist_ok=True);
pairs_list= [];
for ii,dd in enumerate(dateList):
    referenceDate = dd;
    for jj in range(ii+1, ii+1+numConnections):
        if jj < len(dateList):
            secondaryDate = dateList[jj];
            ifg_pair =  referenceDate+'_'+secondaryDate;
            pairs_list.append(ifg_pair);


In [40]:
#generate interferograms
def generate_isce_xml(ifg_file, amp_file, coh_file, samples):
    outInt = isceobj.Image.createIntImage()
    outInt.setFilename(ifg_file)
    outInt.setWidth(samples)
    outInt.setAccessMode('read')
    outInt.renderHdr()
    outInt.renderVRT()

    outAmp = isceobj.Image.createAmpImage()
    outAmp.setFilename(amp_file)
    outAmp.setWidth(samples)
    outAmp.setAccessMode('read')
    outAmp.renderHdr()
    outAmp.renderVRT()

    outCor = isceobj.Image.createImage()
    outCor.setFilename(coh_file)
    outCor.setWidth(samples)
    outCor.setAccessMode('read')
    outCor.setDataType('FLOAT')
    outCor.renderHdr()
    outCor.renderVRT()

def cat_and_interfere(slc_dir,samples,looks,segments,ifg_dir,pair):
    referenceDate,secondaryDate = pair.split('_');
    int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);os.makedirs(int_dir, exist_ok=True);
    intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
    ampf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.amp');
    cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
    igram = Interferogram(samples, looks);
    int_samples = samples//looks[1];

    if len(segments)>1:
        with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
            for seg in segments:
                ref_img = slc_dir[:-1]+str(seg)+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
                sec_img = slc_dir[:-1]+str(seg)+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
                img0 = ImageReader(ref_img, samples, dtype='complex64')
                img1 = ImageReader(sec_img, samples, dtype='complex64')
                #log.info('... %s * conj(%s)', img0.filename, img1.filename)
                for int_row, amp_row in igram.iterrows(img0, img1):
                    int_row.tofile(fint);
                    amp_row.tofile(famp);
        correlation(intf, ampf, cohf);
        generate_isce_xml(intf, ampf, cohf, int_samples)
    else:
        ref_img = slc_dir+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
        sec_img = slc_dir+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
        with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
            img0 = ImageReader(ref_img, n, dtype='complex64')
            img1 = ImageReader(sec_img, n, dtype='complex64')
            #log.info('... %s * conj(%s)', img0.filename, img1.filename)
            for int_row, amp_row in igram.iterrows(img0, img1):
                    int_row.tofile(fint);
                    amp_row.tofile(famp);
        correlation(intf, ampf, cohf);
        generate_isce_xml(intf, ampf, cohf, int_samples)
with multiprocessing.Pool(processes=12) as pool:
    pool.starmap(cat_and_interfere,[(slc_dir,samples,looks,segments,ifg_dir,pair) for pair in pairs_list]);

API open (R): /scratch/bvarugu/track_14003/Igrams/20101116_20110712/20101116_20110712.int
API open (R): /scratch/bvarugu/track_14003/Igrams/20100511_20101116/20100511_20101116.int
API open (R): /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.int
API close:  /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.int
API close:  /scratch/bvarugu/track_14003/Igrams/20101116_20110712/20101116_20110712.int
API close:  /scratch/bvarugu/track_14003/Igrams/20100511_20101116/20100511_20101116.int
API open (R): /scratch/bvarugu/track_14003/Igrams/20091118_20101116/20091118_20101116.int
API close:  /scratch/bvarugu/track_14003/Igrams/20091118_20101116/20091118_20101116.int
API open (R): /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.amp
API close:  /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.amp
API open (R): /scratch/bvarugu/track_14003/Igrams/20101116_20110712/20101116_20110712.amp
API close:  /scratch

In [ ]:
#unwrap interferograms
def run_cmd(cmd):
    print("Command: {}".format(cmd))
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print("Command failed with error: {}".format(e))
        
def unwrap_snaphu(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair):
    referenceDate,secondaryDate = pair.split('_');
    reference_slc_dir = slc_dir+'/'+referenceDate
    int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);
    intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
    unwf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.unw');
    cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
    cmd = ['unwrap.py -i {} -c {} -u {} -s {}/ -a {} -r {} -d {} -m {}'.format(intf,cohf,unwf,reference_slc_dir,looks[0],looks[1],defo_max,unwrap_method)];
    run_cmd(cmd)
unwrap_method = 'snaphu';
defo_max = 2;
for pair in pairs_list:
    unwrap_snaphu(unwrap_method, defo_max, looks, slc_dir, ifg_dir, pair);
    
#####Currently not working, crashing the instance ########
# with multiprocessing.Pool(processes=12) as pool:
#     pool.starmap(unwrap_snaphu,[(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair) for pair in pairs_list]);
# if __name__ == "__main__":
#     with ProcessPoolExecutor(max_workers=12) as executor:
#         executor.map(unwrap_snaphu, pairs_list)

Command: ['unwrap.py -i /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.int -c /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.coh -u /scratch/bvarugu/track_14003/Igrams/20091118_20100511/20091118_20100511.unw -s /scratch/bvarugu/track_14003/SLC_seg1/20091118/ -a 12 -r 3 -d 2 -m snaphu']

snaphu v1.4.2


In [32]:
#create UAVSAR baselines
baselines_dir = os.path.join(out_dir,'baselines');
os.makedirs(baselines_dir, exist_ok=True);
cmd ='uavsar_baselines.py -s {} -b {}  -m {}'.format(slc_dir,baselines_dir,reference_slc_dir);
print(cmd);
run_cmd(cmd);


uavsar_baselines.py -s /scratch/bvarugu/track_14003/SLC_seg1 -b /scratch/bvarugu/track_14003/baselines  -m /scratch/bvarugu/track_14003/SLC_seg1/20091118
Command: uavsar_baselines.py -s /scratch/bvarugu/track_14003/SLC_seg1 -b /scratch/bvarugu/track_14003/baselines  -m /scratch/bvarugu/track_14003/SLC_seg1/20091118


/home/jovyan/bhuvan_uavsar/uavsar_utils/uavsar_baselines.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import os, imp, sys, glob


/scratch/bvarugu/track_14003/SLC_seg1
/scratch/bvarugu/track_14003/baselines
/scratch/bvarugu/track_14003/SLC_seg1/20091118
reference date was not found. The first acquisition will be considered as the stack reference date.
/scratch/bvarugu/track_14003/SLC_seg1/20091118
/scratch/bvarugu/track_14003/SLC_seg1/20100511
Using entire span of image for estimating baselines
2024-06-09 22:23:01,957 - isce.mroipac.baseline - INFO - Sampling time 0
2024-06-09 22:23:01,971 - isce.mroipac.baseline - INFO - Sampling time 1
2024-06-09 22:23:01,986 - isce.mroipac.baseline - INFO - Sampling time 2
20091118 _ 20100511
/scratch/bvarugu/track_14003/SLC_seg1/20091118
/scratch/bvarugu/track_14003/SLC_seg1/20101116
Using entire span of image for estimating baselines
2024-06-09 22:23:02,045 - isce.mroipac.baseline - INFO - Sampling time 0
2024-06-09 22:23:02,060 - isce.mroipac.baseline - INFO - Sampling time 1
2024-06-09 22:23:02,076 - isce.mroipac.baseline - INFO - Sampling time 2
20091118 _ 20101116
/scrat

In [26]:
#generate geometry files
geometery_dir = os.path.join(out_dir,'geometry'+str(alks)+'x'+str(rlks));
cmd = 'topo.py -a {} -r {} -d {} -m {} -o {} -n'.format(alks, rlks,dem_file,reference_slc_dir,geometery_dir);
print(cmd);
run_cmd(cmd);

topo.py -a 12 -r 3 -d /scratch/bvarugu/track_14003/DEM/demLat_N35_N37_Lon_W122_W120.dem.wgs84 -m /scratch/bvarugu/track_14003/SLC_seg1/20091118 -o geometry12x3 -n
Command: topo.py -a 12 -r 3 -d /scratch/bvarugu/track_14003/DEM/demLat_N35_N37_Lon_W122_W120.dem.wgs84 -m /scratch/bvarugu/track_14003/SLC_seg1/20091118 -o geometry12x3 -n
CPU mode
2024-06-09 20:43:50,754 - isce.zerodop.topozero - WARNING - Default Peg heading set to: 2.445241259030772
Polynomial Order: 0 - by - 6 
-48.5732	0.0313894	-1.03308e-05	2.26803e-09	-2.9947e-13	2.11005e-17	-6.05582e-22	
Polynomial Order: 0 - by - 1 
13166.1	1.66551	
GDAL open (R): /scratch/bvarugu/track_14003/DEM/demLat_N35_N37_Lon_W122_W120.dem.wgs84.vrt
API open (WR): geometry12x3/lat.rdr
API open (WR): geometry12x3/lon.rdr
API open (WR): geometry12x3/hgt.rdr
API open (WR): geometry12x3/los.rdr
API open (WR): geometry12x3/incLocal.rdr
API open (WR): geometry12x3/shadowMask.rdr
 Initializing biquintic interpolator
 Orbit interpolation method: hermit

In [37]:
#create referenceShelve dir
reference_dir = os.path.join(out_dir,'referenceShelve');
reference_ifg = sorted(glob.glob(ifg_dir+'/'+reference_date+'_*'))[0];
print('Reference interferogram is:',reference_ifg)
cmd ='cp -r {}/referenceShelve {}'.format(reference_ifg,reference_dir)
print(cmd);
run_cmd(cmd);

Reference interferogram is: /scratch/bvarugu/track_14003/Igrams/20091118_20100511
cp -r /scratch/bvarugu/track_14003/Igrams/20091118_20100511/referenceShelve /scratch/bvarugu/track_14003/referenceShelve
Command: cp -r /scratch/bvarugu/track_14003/Igrams/20091118_20100511/referenceShelve /scratch/bvarugu/track_14003/referenceShelve


In [30]:
#Prepare .rsc files for unwrapped interferograms to be loaded into MintPy
cmd = 'prep_isce.py -f {}/*/{} -m {}/data -b {}  -g {}'.format(ifg_dir,'*.unw',reference_dir,baselines_dir,geometery_dir)
print(cmd);
run_cmd(cmd);

prep_isce.py -f /scratch/bvarugu/track_14003/Igrams/*/*.unw -m /scratch/bvarugu/track_14003/referenceShelve/data -b baselines  -g geometry12x3
Command: prep_isce.py -f /scratch/bvarugu/track_14003/Igrams/*/*.unw -m /scratch/bvarugu/track_14003/referenceShelve/data -b baselines  -g geometry12x3
extract metadata from ISCE/stripmapStack shelve file: /scratch/bvarugu/track_14003/referenceShelve/data
extract metadata from geometry files: ['hgt.rdr', 'lat.rdr', 'lon.rdr', 'los.rdr']
writing  /scratch/bvarugu/track_14003/referenceShelve/data.rsc
preparing RSC file for geometry files
write file: /scratch/bvarugu/track_14003/geometry12x3/hgt.rdr.rsc
write file: /scratch/bvarugu/track_14003/geometry12x3/lat.rdr.rsc
write file: /scratch/bvarugu/track_14003/geometry12x3/lon.rdr.rsc
write file: /scratch/bvarugu/track_14003/geometry12x3/los.rdr.rsc
write file: /scratch/bvarugu/track_14003/geometry12x3/shadowMask.rdr.rsc
write file: /scratch/bvarugu/track_14003/geometry12x3/incLocal.rdr.rsc
read perp

In [35]:

mintpy_dir= os.path.join(out_dir,'mintpy');
os.makedirs(mintpy_dir, exist_ok=True);
config_file = Path(mintpy_dir)/('smallbaselineApp.cfg')
config_file_parameters = """
####mintpy parameters
mintpy.load.processor = isce
mintpy.compute.numWorker = auto
##path to interferograms
mintpy.load.autoPath = no
mintpy.load.metaFile       = {out_dir}/referenceShelve/data
mintpy.load.baselineDir    = {out_dir}/baselines  
mintpy.load.unwFile        = {out_dir}/Igrams/*/*.unw_snaphu.unw  
mintpy.load.corFile        = {out_dir}/Igrams/*/*.coh
mintpy.load.connCompFile   = {out_dir}/Igrams/*/*.unw_snaphu.conncomp  
#mintpy.load.intFile        = {out_dir}/Igrams/*/*.int  
###path to geometry files
mintpy.load.demFile = {out_dir}/{geometery_dir}/hgt.rdr
mintpy.load.lookupYFile = {out_dir}/{geometery_dir}/lat.rdr
mintpy.load.lookupXFile = {out_dir}/{geometery_dir}/lon.rdr
mintpy.load.incAngleFile = {out_dir}/{geometery_dir}/los.rdr
mintpy.load.azAngleFile = {out_dir}/{geometery_dir}/los.rdr
mintpy.load.shadowMaskFile = {out_dir}/{geometery_dir}/shadowMask.msk
""".format(out_dir=out_dir,geometery_dir=geometery_dir)
config_file.write_text(config_file_parameters);
print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

MintPy config file:
    /scratch/bvarugu/track_14003/mintpy/smallbaselineApp.cfg:

####mintpy parameters
mintpy.load.processor = isce
mintpy.compute.numWorker = auto
##path to interferograms
mintpy.load.autoPath = no
mintpy.load.metaFile       = /scratch/bvarugu/track_14003/referenceShelve/data
mintpy.load.baselineDir    = /scratch/bvarugu/track_14003/baselines  
mintpy.load.unwFile        = /scratch/bvarugu/track_14003/Igrams/*/*.unw_snaphu.unw  
mintpy.load.corFile        = /scratch/bvarugu/track_14003/Igrams/*/*.coh
mintpy.load.connCompFile   = /scratch/bvarugu/track_14003/Igrams/*/*.unw_snaphu.conncomp  
#mintpy.load.intFile        = /scratch/bvarugu/track_14003/Igrams/*/*.int  
###path to geometry files
mintpy.load.demFile = /scratch/bvarugu/track_14003/geometry12x3/hgt.rdr
mintpy.load.lookupYFile = /scratch/bvarugu/track_14003/geometry12x3/lat.rdr
mintpy.load.lookupXFile = /scratch/bvarugu/track_14003/geometry12x3/lon.rdr
mintpy.load.incAngleFile = /scratch/bvarugu/track_14003/ge

In [36]:
os.chdir(mintpy_dir);
cmd = 'smallbaselineApp.py ' + 'smallbaselineApp.cfg' + ' --dostep load_data'
print(cmd);
run_cmd(cmd);

smallbaselineApp.py smallbaselineApp.cfg --dostep load_data
Command: smallbaselineApp.py smallbaselineApp.cfg --dostep load_data


/home/jovyan/.local/envs/isce/lib/python3.9/site-packages/mintpy/load_data.py:669: UserWarning: No input metadata file found: /scratch/bvarugu/track_14003/referenceShelve/data
  warnings.warn('No input metadata file found: {}'.format(iDict['mintpy.load.metaFile']))
/home/jovyan/.local/envs/isce/lib/python3.9/site-packages/mintpy/load_data.py:707: UserWarning: prep_isce.py failed. Assuming its result exists and continue...
  warnings.warn('prep_isce.py failed. Assuming its result exists and continue...')


MintPy version 1.6.0, date 2024-05-09
--RUN-at-2024-06-09 22:28:16.545711--
Current directory: /scratch/bvarugu/track_14003/mintpy
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_ionosphere', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Go to work directory: /scratch/bvarugu/track_14003/mintpy
obsolete default template detected, update to the latest version.
    mintpy.load.processor: auto --> isce
    mintpy.load.autoPath: auto --> no
    mintpy.load.metaFile: auto --> /scratch/bvarugu/track_14003/referenceShelve/data
    mintpy.load.baselineDir: auto --> /scratch/bvarugu/track_14003/baselines
    mintpy.load.unwFile: auto --> /scratch/bvarugu/track_14003/Igrams/*/*.u